import socket
import threading
import os

In [2]:
client = socket.socket()

In [3]:
client.connect(('localhost', 9999))

In [4]:
stop_event = threading.Event()
file_permission = None

In [5]:
def receive():
    global file_permission
    
    while not stop_event.is_set():
        msg = client.recv(1024)

        if not msg:
            print("Server closed connection.")
            stop_event.set()
            break

        decoded = msg.decode()

        if decoded == "NICK":
            client.send(nickname.encode())

        elif decoded == "ACCEPTED":
            print("Server: File accepted.")
            file_permission = True
        
        elif decoded.startswith("REJECTED:"):
            print(f"Server: File type {decoded.split(':')[1]} not allowed!")
            file_permission = False

        elif decoded.startswith("FILE_FROM_SERVER:"):
            _, filename, filesize = decoded.split(":")
            filesize = int(filesize)

            with open("received_by_" + nickname + "_" + filename, "wb") as f:
                bytes_read = 0
                while bytes_read < filesize:
                    data = client.recv(1024)
                    f.write(data)
                    bytes_read += len(data)

            print(f"Received file: {filename}")

        else:
            print(decoded)

In [6]:
def write():
    global file_permission

    while not stop_event.is_set():
        msg = input()

        if msg.lower() == "exit":
            client.send("exit".encode())
            stop_event.set()
            break

        if msg.lower() == "file":
            filepath = input("Enter file path: ")
        
            if not os.path.exists(filepath):
                print("File does not exist.")
                continue
        
            filename = os.path.basename(filepath)
            filesize = os.path.getsize(filepath)
        
            file_permission = None
        
            client.send(f"FILE:{filename}:{filesize}".encode())
        
            while file_permission is None:
                pass
        
            if file_permission is False:
                continue
        
            with open(filepath, "rb") as f:
                while True:
                    bytes_read = f.read(1024)
                    if not bytes_read:
                        break
                    client.send(bytes_read)
        
            print("File sent successfully.")

        else:
            full_msg = f"{nickname}: {msg}"
            client.send(full_msg.encode())

In [7]:
nickname = input("Choose your nickname: ")

receive_thread = threading.Thread(target=receive)
write_thread = threading.Thread(target=write)

receive_thread.start()
write_thread.start()

receive_thread.join()
write_thread.join()

client.close()

Choose your nickname:  c3


c3 joined the chat!
Connected to the server!
c1: hi
Received file: 1770918764658.jpg


 hey
 file
Enter file path:  D:\Downloads\kaggle.json


Server: File type .json not allowed!
c1: ok
c2: set
c1 left the chat!


 exit


Server closed connection.
